<a href="https://colab.research.google.com/github/259mit/AI/blob/master/CS50/Learning/J002_Learning_Nim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import random
import time

In [2]:
class Nim():

    def __init__(self, initial=[1, 3, 5, 7]):
        self.piles = initial.copy()
        self.player = 0
        self.winner = None

    @classmethod
    def available_actions(cls, piles):
        actions = set()
        for i, pile in enumerate(piles):
            for j in range(1, pile + 1):
                actions.add((i, j))
        return actions

    @classmethod
    def other_player(cls, player):
        return 0 if player == 1 else 1

    def switch_player(self):
        self.player = Nim.other_player(self.player)

    def move(self, action):
        pile, count = action

        # Check for errors
        if self.winner is not None:
            raise Exception("Game already won")
        elif pile < 0 or pile >= len(self.piles):
            raise Exception("Invalid pile")
        elif count < 1 or count > self.piles[pile]:
            raise Exception("Invalid number of objects")

        # Update pile
        self.piles[pile] -= count
        self.switch_player()

        # Check for a winner
        if all(pile == 0 for pile in self.piles):
            self.winner = self.player

In [3]:
def train(n):
    player = NimAI()

    # Play n games
    for i in range(n):
        print(f"Playing training game {i + 1}")
        game = Nim()

        # Keep track of last move made by either player
        last = {
            0: {"state": None, "action": None},
            1: {"state": None, "action": None}
        }

        # Game loop
        while True:

            # Keep track of current state and action
            state = game.piles.copy()
            action = player.choose_action(game.piles)

            # Keep track of last state and action
            last[game.player]["state"] = state
            last[game.player]["action"] = action

            # Make move
            game.move(action)
            new_state = game.piles.copy()

            # When game is over, update Q values with rewards
            if game.winner is not None:
                player.update(state, action, new_state, -1)
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    1
                )
                break

            # If game is continuing, no rewards yet
            elif last[game.player]["state"] is not None:
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    0
                )

    print("Done training")

    # Return the trained AI
    return player

In [4]:
def play(ai, human_player=None):
    # If no player order set, choose human's order randomly
    if human_player is None:
        human_player = random.randint(0, 1)

    # Create new game
    game = Nim()

    # Game loop
    while True:

        # Print contents of piles
        print()
        print("Piles:")
        for i, pile in enumerate(game.piles):
            print(f"Pile {i}: {pile}")
        print()

        # Compute available actions
        available_actions = Nim.available_actions(game.piles)
        time.sleep(1)

        # Let human make a move
        if game.player == human_player:
            print("Your Turn")
            while True:
                pile = int(input("Choose Pile: "))
                count = int(input("Choose Count: "))
                if (pile, count) in available_actions:
                    break
                print("Invalid move, try again.")

        # Have AI make a move
        else:
            print("AI's Turn")
            pile, count = ai.choose_action(game.piles, epsilon=False)
            print(f"AI chose to take {count} from pile {pile}.")

        # Make move
        game.move((pile, count))

        # Check for winner
        if game.winner is not None:
            print()
            print("GAME OVER")
            winner = "Human" if game.winner == human_player else "AI"
            print(f"Winner is {winner}")
            return

In [5]:
class NimAI():

    def __init__(self, alpha=0.5, epsilon=0.1):
        self.q = dict()
        self.alpha = alpha
        self.epsilon = epsilon

    def update(self, old_state, action, new_state, reward):
        old = self.get_q_value(old_state, action)
        best_future = self.best_future_reward(new_state)
        self.update_q_value(old_state, action, old, reward, best_future)

    def get_q_value(self, state, action):
        if (tuple(state), action) not in self.q:
            return 0
        
        return self.q[tuple(state), action]

    def update_q_value(self, state, action, old_q, reward, future_rewards):
        self.q[tuple(state), action] = old_q + self.alpha * (reward + future_rewards - old_q)

    def best_future_reward(self, state):
        best_reward = 0

        for available_action in Nim.available_actions(state):
            if self.get_q_value(state, available_action) > best_reward:
                best_reward = self.get_q_value(state, available_action)

        return best_reward

    def choose_action(self, state, epsilon=True):
        available_actions = Nim.available_actions(state)

        # Choose random available action
        if epsilon and random.random() <= self.epsilon:
            return random.sample(available_actions, 1)[0]
        
        # Otherwise choose best available action
        best_reward = -math.inf

        for available_action in available_actions:
            if self.get_q_value(state, available_action) > best_reward:
                best_reward = self.get_q_value(state, available_action)
                best_action = available_action

        return best_action

In [6]:
ai = train(10000)
play(ai)

Streaming output truncated to the last 5000 lines.
Playing training game 5054
Playing training game 5055
Playing training game 5056
Playing training game 5057
Playing training game 5058
Playing training game 5059
Playing training game 5060
Playing training game 5061
Playing training game 5062
Playing training game 5063
Playing training game 5064
Playing training game 5065
Playing training game 5066
Playing training game 5067
Playing training game 5068
Playing training game 5069
Playing training game 5070
Playing training game 5071
Playing training game 5072
Playing training game 5073
Playing training game 5074
Playing training game 5075
Playing training game 5076
Playing training game 5077
Playing training game 5078
Playing training game 5079
Playing training game 5080
Playing training game 5081
Playing training game 5082
Playing training game 5083
Playing training game 5084
Playing training game 5085
Playing training game 5086
Playing training game 5087
Playing training game 5088
Play